In [1]:
import os
import cv2
import copy
import numpy as np
import matplotlib.pyplot as plt


In [10]:
# 椭圆拟合及筛选
def ellipse_fit_filter(image, contours):
    """
    Args:
        image (_type_): init image
        contours (_type_): canny edge detection
    """

    # save circle and bounding box
    result = []

    # detect circle
    processed_centers = []

    for contour in contours:
        # 如果轮廓点数不足 5，则跳过
        if len(contour) < 5:
            print('contour is too short')
            continue

        # 根据圆的面积筛选
        area = cv2.contourArea(contour)
        # if area < 3000:
        #     continue

        # 根据圆形度筛选   
        perimeter = cv2.arcLength(contour, True)
        circularity = 4 * np.pi * (area / (perimeter * perimeter))
        if circularity < 0.8:
            continue

        # 计算轮廓的几何矩
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = M['m10'] / M['m00']
            cy = M['m01'] / M['m00']
        else:
            continue

        # 合并相近的圆心
        close_enough = False
        for center in processed_centers:
            if np.linalg.norm(np.array(center) - np.array((cx, cy))) < 10:  # 设定距离阈值
                close_enough = True
                break

        if close_enough:
            continue  # 如果相近则跳过

        # 拟合椭圆
        ellipse = cv2.fitEllipse(contour)
        w, h = ellipse[1][0], ellipse[1][1]

        # 计算 bounding box 的左上角坐标
        top_left_x = int(cx - w / 2)
        top_left_y = int(cy - h / 2)

        # 在图像上绘制 bounding box
        cv2.rectangle(image, (top_left_x, top_left_y), (top_left_x + int(w), top_left_y + int(h)), (255, 0, 0), 2)

        # 标注圆心
        int_center = (int(cx), int(cy))
        cv2.circle(image, int_center, 1, (0, 0, 255), -1)

        # 记录处理后的圆心
        processed_centers.append((cx, cy))
        result.append((cx, cy, w, h))
        # 输出bounding box的宽高和圆心坐标
        # print(f"Center: {center}, Bounding Box (w, h): ({w}, {h})")

    # 显示结果
    # cv2.imwrite('data/result.png', image)
    image = cv2.resize(image, (0, 0), fx=0.3, fy=0.3)
    cv2.imshow('Fitted Ellipses', image)
    cv2.waitKey(1000)
    cv2.destroyAllWindows()

    return result

In [4]:
# 单张图像提取圆心测试
# img_path = 'data/circle_images/valid_accuracy.png'
img_path = 'synthetic_subpixel_circles_with_noise_sharp.png'

img = cv2.imread(img_path)

# 转为灰度图像
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 高斯模糊去噪
blurred = cv2.GaussianBlur(gray, (5, 5), 1.5)

# 使用 Canny 进行边缘检测
edges = cv2.Canny(blurred, 50, 100)

# 查找轮廓
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

center_bounding_box = ellipse_fit_filter(img, contours)
print(len(center_bounding_box))
for ele in center_bounding_box:
    print(ele[0], ele[1])

# 在原图上绘制所有轮廓，颜色为绿色，线条宽度为2
cv2.drawContours(img, contours, -1, (0, 255, 0), 2)

img = cv2.resize(img, (0, 0), fx=0.3, fy=0.3)

# 显示图像
cv2.imshow('Contours', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


15
1836.40815429074 2386.634941575575
784.0945345571902 2300.900247286139
390.99723374827107 2299.7812984617963
3052.3795023126345 1906.7196375128187
2937.1032604140473 1771.1167127650656
556.3808536917247 1597.7196926877261
217.72332057156007 1562.223958660223
3271.3799267399263 1415.2740554683412
408.38214240863994 1382.4814768303404
494.6162428048142 1259.29220303506
1343.9937751509053 1011.9431170355465
1443.3140627288515 487.75467118615694
1676.4347771500313 295.8494245658087
2781.4542257942985 189.62777847544896
2314.6514647067443 53.80178821952803


In [11]:
# 文件夹图像提取圆心测试
# img_path = 'data/circle_images/valid_accuracy.png'
img_path = 'E:/INFJ/1104-dis/circle'

for filename in os.listdir(img_path):
    # 检查文件扩展名
    if filename.endswith('.bmp'):
        # 构建文件的完整路径
        file_path = os.path.join(img_path, filename)

        img = cv2.imread(file_path)

        # 转为灰度图像
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # 高斯模糊去噪
        blurred = cv2.GaussianBlur(gray, (5, 5), 1.5)

        # 使用 Canny 进行边缘检测
        edges = cv2.Canny(gray, 0, 255)

        # 查找轮廓
        contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        center_bounding_box = ellipse_fit_filter(img, contours)
        print(len(center_bounding_box))
        for ele in center_bounding_box:
            print(ele[0], ele[1])
# print(contours)

# 在原图上绘制所有轮廓，颜色为绿色，线条宽度为2
# cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
# 
# img = cv2.resize(img, (0, 0), fx=0.3, fy=0.3)

# 显示图像
# cv2.imshow('Contours', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()




2
1583.1681699522908 452.43622288234764
2945.708436813879 421.4525429219307
2
496.5341977477012 503.23146330543784
1858.2115926801505 479.4151650468435
2
1583.083678928565 450.1405001705535
2946.010117922405 432.220055810604
contour is too short
2
496.60253491981376 502.2152440075875
1857.9561578947369 486.9094736842105
2
1578.8057631310144 454.26464526996756
2941.710613373827 434.1021205054504
2
489.3176800300762 531.6736901466215
1840.0855096360121 467.6487243586334
2
1572.9841074705578 479.38808416626574
2932.4121424401633 412.8790499124343
2
489.4133646914464 560.8823729911209
1832.4904200245503 442.73695540018855
2
1571.4083608712644 532.5346977934122
2920.955420076527 372.89950751542443
1
487.5054418287273 608.7853633572159
2
1570.8938941607134 556.1459643885275
2915.5430750658475 356.98397717295876
1
486.86635260381996 631.6608804000887
2
1571.7043157388096 599.0293955113464
2904.750269362114 329.1503497142022
2
490.8926133155382 684.7812569512174
1807.1243712362734 364.88691108

KeyboardInterrupt: 